In [21]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import spikeinterface as si
import matplotlib.pyplot as plt
import os
from matplotlib.backends.backend_pdf import PdfPages

In [22]:
select_neurons = ['Neuron_1',  "Neuron_4",
                  'Neuron_5', 'Neuron_6', 'Neuron_7', 'Neuron_8',
                  'Neuron_12', 'Neuron_15', "Neuron_16", "Neuron_17",
                  "Neuron_20"]

spike_inf = pd.read_csv("/media/ubuntu/sda/data/mouse11/output/analysis_natural_image/02_merge_new/all_date/spike_inf_refine.tsv", sep = '\t')
cluster_inf = pd.read_csv("/media/ubuntu/sda/data/mouse11/output/analysis_natural_image/02_merge_new/all_date/cluster_inf_refine.tsv", sep = '\t')

In [23]:
trigger_time = pd.read_csv("/media/ubuntu/sda/data/mouse11/output/analysis_natural_image/02_merge_new/all_date/trigger_time_refine.csv").iloc[:, 1:]

import pickle
with open('/media/ubuntu/sda/data/mouse11/output/analysis_natural_image/02_merge_new/all_date/outlier.pkl', 'rb') as f:
    outlier = pickle.load(f)


In [24]:
outlier = [item for item in outlier if not any(e in item for e in ['021722', '030122','112822'])]

In [25]:
for i in outlier:
    date = i.split("_")[0]
    image = i.split("_")[1] + "_" + i.split("_")[2]
    trigger_time = trigger_time[~((trigger_time['date'] == int(date)) & (trigger_time['order'] == image))]

In [26]:
# expand time to -0.5s - 1.5s
trigger_time['expand_start'] = trigger_time['start'] - 5000
trigger_time['expand_end'] = trigger_time['end'] + 10000


In [27]:
date_order = ['021722', '030122', '032322', '042322', '052322', '052422', '062422', '072422', '082422', '092222', '102522', '112822',
              '122322', '012123', '022423', '032323', '042323', '052423', '062323', '072123']

In [ ]:
with PdfPages("raster_overall.pdf") as pdf:
    for neuron in spike_inf['Neuron'].unique():
        neuron_df = spike_inf[spike_inf['Neuron'] == neuron]

        fig, ax = plt.subplots(figsize = (12, 7.5))

        for index, row in trigger_time.iterrows():
            date = row['date']
            image_order = row['order']
            start = row['expand_start'] 
            end = row['expand_end']
            
            filtered_spikes = neuron_df[(neuron_df['date'] == date) & 
                                        (neuron_df['time'] >= start) & 
                                        (neuron_df['time'] <= end)]
            
            if not filtered_spikes.empty:
                ax.plot(filtered_spikes['time'] - start, [index] * len(filtered_spikes), marker='|', mew=1, markersize=3, ls='', color='k')

        ax.set_yticks([])
        #ax.axvspan(1 + (5000 - 1) * 0.5, 1 + (5000 - 1), color='gray', alpha=0.3)

        ax.set_xlabel('time (s)')
        ax.set_ylabel("")
        ax.set_title(f'Raster Plot for {neuron}')

        pdf.savefig(fig)
        plt.close()

In [29]:
mean_spike_rate_dict = {}

with PdfPages("peth_lineplot_all.pdf") as pdf:
    for neuron in spike_inf['Neuron'].unique():
        plt.figure(figsize=(4, 2.5))
        spike_rates_all = np.empty((99))

        for image in range(1, 19):
            neuron_df = spike_inf[spike_inf['Neuron'] == neuron]

            dates = trigger_time[trigger_time['image'] == image]['date'].unique()
            date_indices = np.arange(len(dates))  

            spike_rates = []
            std_rates = []

            for date in dates:
                trigger_time_temp = trigger_time[(trigger_time['image'] == image) & (trigger_time['date'] == date)]
                all_spikes = []

                for index, row in trigger_time_temp.iterrows():
                    start = row['expand_start'] 
                    end = row['expand_end']

                    filtered_spikes = neuron_df[(neuron_df['date'] == date) &
                                                (neuron_df['time'] >= start) &
                                                (neuron_df['time'] <= end)]

                    if not filtered_spikes.empty:
                        relative_spikes = filtered_spikes['time'] - start
                        all_spikes.extend(relative_spikes)

                if all_spikes:
                    time_bins = np.linspace(0, max(all_spikes), 100)
                    spike_counts, _ = np.histogram(all_spikes, bins=time_bins)
                    spike_rate = spike_counts / (time_bins[1] - time_bins[0])
                    spike_rates.append(spike_rate)
                spike_rates_all = np.vstack((spike_rates_all, spike_rates))

        mean_spike_rate = np.mean(spike_rates, axis=0)
        std_spike_rate = np.std(spike_rates, axis=0)

        plt.plot(time_bins[:-1], mean_spike_rate, color='k', linewidth = 2)
        plt.plot(time_bins[:-1], mean_spike_rate - std_spike_rate, color='lightgray', alpha=0.7, linewidth = 1)
        plt.plot(time_bins[:-1], mean_spike_rate + std_spike_rate, color='lightgray', alpha=0.7, linewidth = 1)
        plt.title(neuron)
        plt.xticks([])
        plt.yticks([])
        pdf.savefig()
        plt.close()

In [16]:
with PdfPages("raster_split_image.pdf") as pdf:
    for neuron in spike_inf['Neuron'].unique():
        neuron_df = spike_inf[spike_inf['Neuron'] == neuron]


        fig, axs = plt.subplots(9, 13, figsize=(25, 15), sharex=True, sharey=True)
        fig.subplots_adjust(hspace=0.5, wspace=0.5)

        for image in range(1, 2):  
            ax = axs[(image - 1) // 13, (image - 1) % 13]  
            trigger_time_temp = trigger_time[trigger_time['image'] == image]

            for index, row in trigger_time_temp.iterrows():
                date = row['date']
                image_order = row['order']
                start = row['start']
                end = row['end']

                filtered_spikes = neuron_df[(neuron_df['date'] == date) & 
                                            (neuron_df['time'] >= start) & 
                                            (neuron_df['time'] <= end)]

                if not filtered_spikes.empty:
                    ax.plot(filtered_spikes['time'] - start, [index] * len(filtered_spikes), 
                            marker='|', mew=1, markersize=3, ls='', color='k')


            ax.set_xticks([])
            ax.set_yticks([])

        pdf.savefig(fig)
        plt.close()

In [20]:
mean_spike_rate_dict = {}

with PdfPages("peth_lineplot.pdf") as pdf:
    for image in range(1, 118):
        fig, axes = plt.subplots(4, 5, figsize=(15, 9))
        axes = axes.flatten()
        num = 0
        image_mean_spike_rate_data = []
        for neuron in spike_inf['Neuron'].unique():
            ax = axes[num]
            neuron_df = spike_inf[spike_inf['Neuron'] == neuron]

            dates = trigger_time[trigger_time['image'] == image]['date'].unique()
            date_indices = np.arange(len(dates))  

            spike_rates = []
            std_rates = []

            for date in dates:
                trigger_time_temp = trigger_time[(trigger_time['image'] == image) & (trigger_time['date'] == date)]
                all_spikes = []

                for index, row in trigger_time_temp.iterrows():
                    start = row['start'] - 5000
                    end = row['end'] + 10000

                    filtered_spikes = neuron_df[(neuron_df['date'] == date) &
                                                (neuron_df['time'] >= start) &
                                                (neuron_df['time'] <= end)]

                    if not filtered_spikes.empty:
                        relative_spikes = filtered_spikes['time'] - start
                        all_spikes.extend(relative_spikes)

                if all_spikes:
                    time_bins = np.linspace(0, max(all_spikes), 100)
                    spike_counts, _ = np.histogram(all_spikes, bins=time_bins)
                    spike_rate = spike_counts / (time_bins[1] - time_bins[0])
                    spike_rates.append(spike_rate)

            mean_spike_rate = np.mean(spike_rates, axis=0)
            std_spike_rate = np.std(spike_rates, axis=0)

            image_mean_spike_rate_data.append({
                'Neuron': neuron,
                'Mean_Spike_Rate': mean_spike_rate
            })

            ax.plot(time_bins[:-1], mean_spike_rate, color='k', linewidth = 2)
            ax.plot(time_bins[:-1], mean_spike_rate - std_spike_rate, color='lightgray', alpha=0.7, linewidth = 1)
            ax.plot(time_bins[:-1], mean_spike_rate + std_spike_rate, color='lightgray', alpha=0.7, linewidth = 1)
            #ax.set_ylim(- np.mean(mean_spike_rate) , 2 *np.mean(mean_spike_rate))
            #ax.fill_between(time_bins[:-1], mean_spike_rate - std_spike_rate, mean_spike_rate + std_spike_rate, color='lightgray', alpha=0.5, label='Std Deviation')
            quarter = (time_bins[-1] - time_bins[0]) * 0.25 + time_bins[0]
            three_quarter = (time_bins[-1] - time_bins[0]) * 0.75 + time_bins[0]
            ax.annotate("", xy=(quarter, np.max(mean_spike_rate + std_spike_rate) * 0.9), xytext=(quarter, np.max(mean_spike_rate + std_spike_rate)),
                        arrowprops=dict(arrowstyle="-|>", color="black"))
            ax.annotate("", xy=(three_quarter, np.max(mean_spike_rate + std_spike_rate) * 0.9), xytext=(three_quarter, np.max(mean_spike_rate + std_spike_rate)),
                        arrowprops=dict(arrowstyle="-|>", color="black"))
            ax.set_xlabel('')
            ax.set_ylabel('')
            ax.set_title(f'{neuron}', fontsize=10)
            ax.legend()

            ax.set_xticks([])
            ax.set_yticks([])
            ax.grid(False)
            ax.legend().set_visible(False)
            num += 1
        axes[19].axis('off')
        axes[18].axis('off')
        fig.suptitle(f"Image_{image}", fontsize=20)
        fig.text(0.5, 0.04, 'Time', ha='center', fontsize=14)
        fig.text(0.06, 0.5, 'Spike Rate (Hz)', va='center', rotation='vertical', fontsize=14)
        pdf.savefig(fig)
        plt.close()

        mean_spike_rate_dict[image] = pd.DataFrame(image_mean_spike_rate_data)

In [18]:
for image in range(1, 118):
    mean_spike_rate_dict[image]['image'] = image

In [19]:
mean_spike_rate = pd.DataFrame()
for image in range(1, 118):
    mean_spike_rate = pd.concat((mean_spike_rate, mean_spike_rate_dict[image]), axis=0)

In [20]:
expanded_data = [list(row) for row in mean_spike_rate['Mean_Spike_Rate']]
expanded_df = pd.DataFrame(expanded_data)
expanded_df.columns = [f'Mean_Spike_Rate_{i}' for i in range(99)]
mean_spike_rate.index = expanded_df.index
mean_spike_rate = pd.concat([mean_spike_rate.drop(columns=['Mean_Spike_Rate']), expanded_df], axis=1)

In [21]:
mean_spike_rate = pd.read_csv("/media/ubuntu/sda/data/mouse6/output/03_single_neuron/mean_spike_rate.csv").iloc[:, 1:102]

In [22]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

pca = PCA(n_components=20)
principal_components = pca.fit_transform(mean_spike_rate.iloc[:, 2:])

In [23]:
pca.explained_variance_ratio_

array([0.66993146, 0.15965338, 0.06359484, 0.020278  , 0.01274256,
       0.00965172, 0.00766983, 0.00375075, 0.00299845, 0.00240472,
       0.00218447, 0.00157761, 0.00154862, 0.00112225, 0.00104527,
       0.00096944, 0.00090964, 0.00085789, 0.00079868, 0.00077563])

In [24]:
import umap
reducer = umap.UMAP()
embedding = reducer.fit_transform(principal_components)

In [25]:
principal_components = pd.DataFrame(principal_components)
principal_components.columns = [f'PC_{i}' for i in range(1, 21)]
mean_spike_rate = pd.concat((mean_spike_rate, principal_components), axis=1)

In [38]:
from matplotlib.colors import LinearSegmentedColormap

colors = ["#FF0000", "#FF7F00", "#FFFF00", "#00FF00", "#0000FF", "#4B0082", "#9400D3"]
cmap = LinearSegmentedColormap.from_list("rainbow", colors)
num_colors = 18
rainbow_colors = cmap(np.linspace(0, 1, num_colors))

In [ ]:
with PdfPages("reduced_dim.pdf") as pdf:
    plt.figure(figsize=(5, 5))
    sns.scatterplot(x = principal_components['PC_1'], y = principal_components['PC_2'], s=25, hue = mean_spike_rate['Neuron'])
    plt.xlabel('PC_1')
    plt.ylabel('PC_2')
    plt.xticks([])
    plt.yticks([])
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(5, 5))
    sns.scatterplot(x = principal_components['PC_1'], y = principal_components['PC_2'], s=25, hue = mean_spike_rate['Neuron'], legend=False)
    plt.xlabel('PC_1')
    plt.ylabel('PC_2')
    plt.xticks([])
    plt.yticks([])
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(5, 5))
    sns.scatterplot(x = principal_components['PC_1'], y = principal_components['PC_2'], s=25, hue = mean_spike_rate['image'], legend=False)
    plt.xlabel('PC_1')
    plt.ylabel('PC_2')
    plt.xticks([])
    plt.yticks([])
    pdf.savefig()
    plt.close()

In [52]:
with PdfPages("reduced_dim_Neuron.pdf") as pdf:
    max = mean_spike_rate['PC_1'].max()
    min = mean_spike_rate['PC_1'].min()
    for neuron in mean_spike_rate['Neuron'].unique():
        temp = mean_spike_rate[mean_spike_rate['Neuron'] == neuron]
        plt.figure(figsize=(5, 5))
        sns.scatterplot(x = temp['PC_1'], y = 1, s=16, hue = temp['image'])
        plt.xlim(min, max)
        plt.xlabel('PC_1')
        plt.ylabel('PC_2')
        plt.xticks([])
        plt.yticks([])
        plt.title(neuron)
        pdf.savefig()
        plt.close()

In [27]:
mean_spike_rate.to_csv("mean_spike_rate.csv")

In [29]:
trend_neuron = ['Neuron_11', "Neuron_15", 'Neuron_17', 'Neuron_18', 'Neuron_19', 'Neuron_20', "Neuron_24", 'Neuron_25']

In [44]:
Image_score = pd.DataFrame()
with PdfPages("reduced_dim_Neuron_trend.pdf") as pdf:
    for neuron in trend_neuron:
        temp = mean_spike_rate[mean_spike_rate['Neuron'] == neuron]
        slope, intercept = np.polyfit(temp['PC_1'], temp['PC_2'], 1)
        y_fit = (temp['PC_1'] * slope + intercept).tolist()

        Image_score = pd.concat((Image_score, pd.DataFrame(y_fit)), axis=1)
        plt.figure(figsize=(4, 4))
        sns.scatterplot(x = temp['PC_1'], y = temp['PC_2'], s=50, hue = temp['image'], legend=False)
        plt.plot(temp['PC_1'], y_fit, color='black')
        plt.xlabel('PC_1')
        plt.ylabel('PC_2')
        plt.xticks([])
        plt.yticks([])
        plt.title(neuron)
        pdf.savefig()
        plt.close()
Image_score.columns = trend_neuron
Image_score.index = range(1, 118)

In [35]:
mean_spike_rate['cluster'] = "1"
mean_spike_rate.loc[mean_spike_rate['Neuron'].isin(trend_neuron), 'cluster'] = "2"

In [36]:
with PdfPages("reduced_dim_cluster.pdf") as pdf:
    plt.figure(figsize=(5, 5))
    sns.scatterplot(x = principal_components['PC_1'], y = principal_components['PC_2'], s=25, hue = mean_spike_rate['cluster'])
    plt.xlabel('PC_1')
    plt.ylabel('PC_2')
    plt.xticks([])
    plt.yticks([])
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(5, 5))
    sns.scatterplot(x = principal_components['PC_1'], y = principal_components['PC_2'], s=25, hue = mean_spike_rate['cluster'], legend=False)
    plt.xlabel('PC_1')
    plt.ylabel('PC_2')
    plt.xticks([])
    plt.yticks([])
    pdf.savefig()
    plt.close()

In [10]:
trigger_time.to_csv("trigger_time_refine.csv")
#spike_inf.to_csv("spike_inf_refine.tsv", sep = "\t")
#cluster_inf.to_csv("cluster_inf_refine.tsv", sep = '\t')

In [143]:
mean_spike_rate.iloc[:, 1:].to_csv('mean_spike_rate.csv')